Adapted from https://users.soe.ucsc.edu/~draper/eBay-Google-2013-parallel-rjags-example.txt
Some helpers and example usage of rjags in parallel mode in R. The output from the foreach functions below give an mcmc object that can be manipulated with the coda package, which is automatically loaded with rjags.

The base code is from: http://andrewgelman.com/2011/07/parallel-jags-rngs/ with some updates and a nice combine function. 

This notebook version comes with instruction on installling the required packages in a non-root environment on Clemson University, and some modifications on the initial source code to deal with the setting of non-variable node. 

To set up JAGS, download the software, and install it into a predefined directory, in this case */home/lngo/software*:
<code>
module load gcc/4.8.1
tar xzf JAGS-4.2.0.tar.gz
cd JAGS-4.2.0
./configure --prefix=/home/lngo/software
make
make install
</code>

Prior to the launching of the Jupyter server, the Linux shell environment needs to load the environment variables PKG_CONFIG_PATH, which points to */home/lngo/software/JAGS-4.2.0/lib/pkgconfig*, and LD_LIBRARY_PATH, which contains */home/lngo/software/JAGS-4.2.0/lib/*. 

This can be accomplished by creating a file called R_jupyter.sh that contains the following lines:
<code>
#!/bin/bash
export PKG_CONFIG_PATH="/home/lngo/software/JAG-4.2.0/lib/pkgconfig"
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/home/lngo/software/JAG-4.2.0/lib"
</code>

Users then can insert a single line into *.bashrc* to enable this environment:
<code>
source R_jupyter.sh
</code>

In [ ]:
# Second, install the following packages into a non-root directory, in this case /home/lngo/R_libs
#  doParallel
#  rjags
#  random

install.packages(c("rjags","doParallel","random"), 
                 lib="/home/lngo/R_libs", 
                 repos='http://cran.us.r-project.org',
                 verbose=TRUE)

In [1]:
# Third, load the installed library. Note that because these libraries are not installed into a default location, the 
# lib parameter within library also needs to be specified

library(doParallel,lib="/home/lngo/R_libs")
library(coda,lib="/home/lngo/R_libs")
library(rjags,lib="/home/lngo/R_libs")
library(random,lib="/home/lngo/R_libs")

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Linked to JAGS 4.2.0
Loaded modules: basemod,bugs


In [2]:
# Function to generate the initial values and random seeds for each model. tau = 0.05 in old version
nb10.inits.1 <- function( ) {
  return( list( mu = 404.59, nu = 5.0, y.new = 400.0, 
    .RNG.name = "lecuyer::RngStream", 
    .RNG.seed = randomNumbers( n = 1, min = 1, max = 1e+06, col = 1 ) ) )
}
# Data to use
nb10.data.1 <- list( 
  y = c( 409., 400., 406., 399., 402., 406., 401., 403., 401., 403.,
         398., 403., 407., 402., 401., 399., 400., 401., 405., 402., 
         408., 399., 399., 402., 399., 397., 407., 401., 399., 401., 
         403., 400., 410., 401., 407., 423., 406., 406., 402., 405., 
         405., 409., 399., 402., 407., 406., 413., 409., 404., 402., 
         404., 406., 407., 405., 411., 410., 410., 410., 401., 402., 
         404., 405., 392., 407., 406., 404., 403., 408., 404., 407., 
         412., 406., 409., 400., 408., 404., 401., 404., 408., 406., 
         408., 406., 401., 412., 393., 437., 418., 415., 404., 401., 
         401., 407., 412., 375., 409., 406., 398., 406., 403., 404.), 
  n = 100 )

# Helper function to combine multiple mcmc lists into a single one. 
mcmc.combine <- function( ... ) {
  return( as.mcmc.list( sapply( list( ... ), mcmc ) ) )
}

In [3]:
# Usage example with some timing to see how much speedup we get with 
# parallel runs. On Chris Severs's machine the time difference is about 3x;
# on my machine it's about 3.6 

# Multi-threaded run (assuming 8 chains)
registerDoParallel(8) 
iters <- 12500

time.1 <- system.time(
  jags.parsamples.8 <- foreach( i = 1:getDoParWorkers( ), .inorder = FALSE, 
    .packages = c( 'rjags', 'random' ), .combine = "mcmc.combine", 
    .multicombine = TRUE ) %dopar% {
      load.module( "lecuyer" )
      model.file <- "nb10-model-1.txt"
      model.jags <- jags.model(data = nb10.data.1,file = model.file,inits = nb10.inits.1)
      result <- coda.samples( model.jags, variable.names = c("mu","sigma","nu","y.new"),n.iter = iters )
    return(result)
  }
)

print( time.1 )
 

   user  system elapsed 
  7.794   0.331   3.205 


In [4]:
# Multi-threaded run (assuming 4 chains)
registerDoParallel(4) 
iters <- 25000

time.2 <- system.time(
  jags.parsamples.4 <- foreach( i = 1:getDoParWorkers( ), .inorder = FALSE, 
    .packages = c( 'rjags', 'random' ), .combine = "mcmc.combine", 
    .multicombine = TRUE ) %dopar% {
      load.module( "lecuyer" )
      model.file <- "nb10-model-1.txt"
      model.jags <- jags.model(data = nb10.data.1,file = model.file,inits = nb10.inits.1)
      result <- coda.samples( model.jags, variable.names = c("mu","sigma","nu","y.new"),n.iter = iters )
    return(result)
  }
)

print( time.2 )
 

   user  system elapsed 
 13.429   0.331   3.668 


In [5]:
# Single-threaded run
iters <- 100000
time.3 <- system.time(
  jags.singlesample <- foreach( i = 1:1, .combine = "mcmc.combine", 
    .multicombine = TRUE) %do% {
    load.module("lecuyer")
    model.jags <- jags.model( data = nb10.data.1, file = "nb10-model-1.txt", 
      inits = nb10.inits.1 )
    result <- coda.samples( model.jags, variable.names = c("mu","sigma","nu", "y.new"), n.iter = iters )
    return( result )
  }
)
print( time.3 )

module lecuyer loaded


Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 100
   Unobserved stochastic nodes: 3
   Total graph size: 112



Warning message:
In jags.model(data = nb10.data.1, file = "nb10-model-1.txt", inits = nb10.inits.1): Unused initial value for "nu" in chain 1

Initializing model



Warning message:
In jags.samples(model, variable.names, n.iter, thin, type = "trace", : Failed to set trace monitor for nu
Variable nu not found


   user  system elapsed 
 11.164   0.025  11.790 


In [6]:
summary( jags.parsamples.8 )

summary( jags.parsamples.4 )
# Iterations = 1001:26000
# Thinning interval = 1 
# Number of chains = 4 
# Sample size per chain = 25000 

# 1. Empirical mean and standard deviation for each variable,
#    plus standard error of the mean:

#          Mean     SD Naive SE Time-series SE
# mu    404.295 0.4719 0.001492       0.001920
# nu      3.637 1.1875 0.003755       0.007821
# sigma   3.863 0.4375 0.001384       0.002462
# y.new 404.295 6.8578 0.021686       0.021686

# 2. Quantiles for each variable:

#          2.5%     25%     50%     75%   97.5%
# mu    403.374 403.978 404.296 404.611 405.224
# nu      2.139   2.804   3.387   4.177   6.638
# sigma   3.078   3.558   3.840   4.141   4.793
# y.new 392.631 401.356 404.282 407.200 416.000

summary( jags.singlesample )
# Iterations = 1001:101000
# Thinning interval = 1 
# Number of chains = 1 
# Sample size per chain = 1e+05 

# 1. Empirical mean and standard deviation for each variable,
#    plus standard error of the mean:

#          Mean     SD Naive SE Time-series SE
# mu    404.299 0.4731 0.001496       0.001917
# nu      3.631 1.1653 0.003685       0.007388
# sigma   3.864 0.4348 0.001375       0.002436
# y.new 404.315 6.6972 0.021178       0.021178

# 2. Quantiles for each variable:

#          2.5%     25%     50%     75%   97.5%
# mu    403.379 403.980 404.295 404.615 405.236
# nu      2.142   2.807   3.387   4.165   6.567
# sigma   3.081   3.560   3.841   4.142   4.782
# y.new 392.684 401.398 404.307 407.196 416.070


Iterations = 1001:13500
Thinning interval = 1 
Number of chains = 8 
Sample size per chain = 12500 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

         Mean     SD Naive SE Time-series SE
mu    404.589 0.6571 0.002078       0.002075
sigma   6.549 0.4742 0.001499       0.001949
y.new 404.581 6.5674 0.020768       0.020718

2. Quantiles for each variable:

         2.5%     25%     50%    75%   97.5%
mu    403.297 404.149 404.586 405.03 405.883
sigma   5.697   6.218   6.523   6.85   7.555
y.new 391.727 400.156 404.577 409.01 417.427



Iterations = 1001:26000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 25000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

         Mean     SD Naive SE Time-series SE
mu    404.592 0.6582 0.002082       0.002076
sigma   6.552 0.4736 0.001498       0.001998
y.new 404.594 6.5903 0.020840       0.020841

2. Quantiles for each variable:

         2.5%     25%     50%     75%   97.5%
mu    403.304 404.151 404.592 405.032 405.886
sigma   5.703   6.222   6.524   6.851   7.561
y.new 391.594 400.179 404.612 409.000 417.520



Iterations = 1001:101000
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 1e+05 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

         Mean     SD Naive SE Time-series SE
mu    404.591 0.6585 0.002082       0.002082
sigma   6.552 0.4754 0.001503       0.001973
y.new 404.575 6.5923 0.020847       0.020847

2. Quantiles for each variable:

         2.5%     25%     50%     75%   97.5%
mu    403.305 404.150 404.590 405.030 405.887
sigma   5.702   6.218   6.523   6.854   7.562
y.new 391.595 400.172 404.589 409.009 417.511


In [ ]:
#plot( jags.parsamples )
#xyplot( jags.parsamples[ , c( "mu", "nu" ) ] )
#autocorr.plot( jags.parsamples )